EasyOCR

In [4]:
import easyocr

reader = easyocr.Reader(['es']) 

path_img = reader.readtext('matricula.jpg')

for (bbox, text, prob) in path_img:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')




Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Texto: 072L HPH
Probabilidad: 0.40
Contenedor: ((11, 1), (126, 38))


In [ ]:
import cv2
import easyocr
from ultralytics import YOLO
import csv
import os

try:
    model_placas = YOLO(r'runs\detect\matricula_yolo11n\weights\best.pt') 
except Exception as e:
    print(f"Error cargando tu modelo de matrículas: {e}")
    exit()


try:
    model_general = YOLO('yolo11n.pt') 
except Exception as e:
    print(f"Error cargando el modelo 'yolo11n.pt': {e}")
    exit()

try:
    reader = easyocr.Reader(['en'], gpu=True) 
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()
    
ALLOWED_CHARACTERS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'


CLASES_GENERALES_INTERES = [0, 2] 
ID_PERSONA = 0 
ID_COCHE = 2    


video_path = r'C:\Users\Alfredo\Downloads\C0142.MP4' 
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: No se pudo abrir el video en {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = 'video_salida_tracking.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

csv_path = 'resultados_tracking.csv'
csv_header = [
    'fotograma', 
    'track_id', 
    'tipo_objeto', 
    'confianza', 
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'
]

try:
    csv_file = open(csv_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except IOError as e:
    print(f"Error abriendo el archivo CSV: {e}")
    exit()

print(f"Guardando video en: {output_path}")
print(f"Guardando datos CSV en: {csv_path}")

frame_counter = 0 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break
        
    frame_counter += 1

    try:
        results_general = model_general.track(
            frame, 
            classes=CLASES_GENERALES_INTERES, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking general: {e}")
        continue

    try:
        results_placas = model_placas.track(
            frame, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking de matrículas: {e}")
        continue

    if results_general.boxes.id is not None:
        boxes = results_general.boxes
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            track_id = int(box.id[0]) 

            if cls == ID_PERSONA:
                nombre_clase = 'persona'
                color = (255, 0, 0) 
            elif cls == ID_COCHE:
                nombre_clase = 'coche'
                color = (0, 0, 255) 
            else:
                continue
            

            label = f"ID: {track_id} {nombre_clase}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


            csv_row = [frame_counter, track_id, nombre_clase, f"{conf:.2f}", x1, y1, x2, y2, '']
            csv_writer.writerow(csv_row)
            
    if results_placas.boxes.id is not None:
        boxes_placas = results_placas.boxes
        for i in range(len(boxes_placas)):
            box = boxes_placas[i]
            x1_placa, y1_placa, x2_placa, y2_placa = map(int, box.xyxy[0])
            conf_placa = float(box.conf[0])
            track_id_placa = int(box.id[0])
            nombre_clase_placa = 'matricula' 
            
            texto_matricula = "" 
            try:
                matricula_img = frame[y1_placa:y2_placa, x1_placa:x2_placa]
                if matricula_img.size > 0:
                    matricula_gray = cv2.cvtColor(matricula_img, cv2.COLOR_BGR2GRAY)
                    ocr_result = reader.readtext(
                        matricula_gray, 
                        detail=0,
                        allowlist=ALLOWED_CHARACTERS,
                        paragraph=False 
                    )
                    if ocr_result:
                        texto_matricula = "".join(ocr_result).upper()
            except Exception as e:
                print(f"Error en OCR: {e}") 


            color_placa = (0, 255, 0) 
            label_placa = f"ID: {track_id_placa} [{texto_matricula}]"
            cv2.rectangle(frame, (x1_placa, y1_placa), (x2_placa, y2_placa), color_placa, 2)
            cv2.putText(frame, label_placa, (x1_placa, y1_placa - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_placa, 2)


            csv_row_placa = [frame_counter, track_id_placa, nombre_clase_placa, f"{conf_placa:.2f}", x1_placa, y1_placa, x2_placa, y2_placa, texto_matricula]
            csv_writer.writerow(csv_row_placa)



    out.write(frame)
    cv2.imshow('Deteccion con Tracking (YOLO + easyOCR)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
out.release() 
csv_file.close() 
cv2.destroyAllWindows()
print("Proceso completado")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Guardando video en: video_salida_tracking.mp4
Guardando datos CSV en: resultados_tracking.csv
Fin del video.
Limpiando y guardando archivos...
¡Proceso completado!


Tesseract

In [ ]:
import time
import cv2
import pytesseract
from ultralytics import YOLO
import csv
import os

print("Iniciando procesamiento...")
start_time = time.time()
pytesseract.pytesseract.tesseract_cmd = r'D:\tesseract\tesseract.exe'




try:
    model_placas = YOLO(r'runs\detect\matricula_yolo11n\weights\best.pt') 
except Exception as e:
    print(f"Error cargando tu modelo de matrículas: {e}")
    exit()

try:
    model_general = YOLO('yolo11n.pt') 
except Exception as e:
    print(f"Error cargando el modelo 'yolo11n.pt': {e}")
    exit()

    
ALLOWED_CHARACTERS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'


CLASES_GENERALES_INTERES = [0, 2] 
ID_PERSONA = 0
ID_COCHE = 2


video_path = r'C:\Users\Alfredo\Downloads\C0142.MP4' 
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video en {video_path}")
    exit()


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = 'video_salida_tracking_tesseract.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


csv_path = 'resultados_tracking_tesseract.csv' 
csv_header = [
    'fotograma', 
    'track_id', 
    'tipo_objeto', 
    'confianza', 
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'
]

try:
    csv_file = open(csv_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except IOError as e:
    print(f"Error abriendo el archivo CSV: {e}")
    exit()

print(f"Guardando video en: {output_path}")
print(f"Guardando datos CSV en: {csv_path}")

frame_counter = 0 


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break
        
    frame_counter += 1

    try:
        results_general = model_general.track(
            frame, 
            classes=CLASES_GENERALES_INTERES, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking general: {e}")
        continue

    try:
        results_placas = model_placas.track(
            frame, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking de matrículas: {e}")
        continue

    if results_general.boxes.id is not None:
        boxes = results_general.boxes
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            track_id = int(box.id[0]) 

            if cls == ID_PERSONA:
                nombre_clase = 'persona'
                color = (255, 0, 0)
            elif cls == ID_COCHE:
                nombre_clase = 'coche'
                color = (0, 0, 255)
            else:
                continue
            
            label = f"ID: {track_id} {nombre_clase}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            csv_row = [frame_counter, track_id, nombre_clase, f"{conf:.2f}", x1, y1, x2, y2, '']
            csv_writer.writerow(csv_row)
            
    if results_placas.boxes.id is not None:
        boxes_placas = results_placas.boxes
        for i in range(len(boxes_placas)):
            box = boxes_placas[i]
            x1_placa, y1_placa, x2_placa, y2_placa = map(int, box.xyxy[0])
            conf_placa = float(box.conf[0])
            track_id_placa = int(box.id[0])
            nombre_clase_placa = 'matricula' 
            
            texto_matricula = "" 
            try:
                matricula_img = frame[y1_placa:y2_placa, x1_placa:x2_placa]
                if matricula_img.size > 0:
                    matricula_gray = cv2.cvtColor(matricula_img, cv2.COLOR_BGR2GRAY)
                    

                    config_tesseract = f'--oem 1 --psm 7 -c tessedit_char_whitelist={ALLOWED_CHARACTERS}'

                    texto_matricula_raw = pytesseract.image_to_string(
                        matricula_gray, 
                        lang='eng',
                        config=config_tesseract
                    )
                    
                    
                    
                    texto_matricula = "".join(texto_matricula_raw.split()).upper()
                    
                    

            except Exception as e:
                print(f"Error en OCR Tesseract: {e}") 

            
            color_placa = (0, 255, 0) 
            label_placa = f"ID: {track_id_placa} [{texto_matricula}]"
            cv2.rectangle(frame, (x1_placa, y1_placa), (x2_placa, y2_placa), color_placa, 2)
            cv2.putText(frame, label_placa, (x1_placa, y1_placa - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_placa, 2)

            
            csv_row_placa = [frame_counter, track_id_placa, nombre_clase_placa, f"{conf_placa:.2f}", x1_placa, y1_placa, x2_placa, y2_placa, texto_matricula]
            csv_writer.writerow(csv_row_placa)


    out.write(frame)
    cv2.imshow('Deteccion con Tracking (YOLO + Tesseract)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

end_time = time.time()
total_time = end_time - start_time
avg_fps = frame_counter / total_time

print(f"\n--- RENDIMIENTO ---")
print(f"Total de fotogramas procesados: {frame_counter}")
print(f"Tiempo total: {total_time:.2f} segundos")
print(f"FPS promedio: {avg_fps:.2f}")

cap.release()
out.release() 
csv_file.close() 
cv2.destroyAllWindows()
print("Proceso completado")

Guardando video en: video_salida_tracking_tesseract.mp4
Guardando datos CSV en: resultados_tracking_tesseract.csv
Fin del video.
Limpiando y guardando archivos...
¡Proceso completado!


In [ ]:
import pandas as pd
import os


# --- 1. CONFIGURACIÓN DE ARCHIVOS ---
archivo_gt = 'ground_truth.csv'
archivo_easyocr = 'resultados_tracking.csv'
archivo_tesseract = 'resultados_tracking_tesseract.csv'

# --- 2. FUNCIÓN DE AYUDA PARA CARGAR DATOS ---
def cargar_datos(archivo_gt, archivo_modelo):
    """Carga el Ground Truth y los resultados del modelo, filtrando solo matrículas."""
    try:
        gt_df = pd.read_csv(archivo_gt)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{archivo_gt}'")
        return None, None
    
    try:
        modelo_df = pd.read_csv(archivo_modelo)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '{archivo_modelo}'")
        return None, None

    # Filtramos los resultados del modelo para quedarnos solo con matrículas
    # y eliminar filas donde el texto_matricula esté vacío (NaN)
    modelo_df = modelo_df[modelo_df['tipo_objeto'] == 'matricula'].copy()
    
    return gt_df, modelo_df

# --- 3. FUNCIÓN PRINCIPAL DE COMPARACIÓN ---
def comparar_modelo(nombre_modelo, gt_df, modelo_df):
    """Compara los resultados de un modelo contra el Ground Truth."""
    
    # Contadores
    aciertos_exactos = 0
    detecciones_incorrectas = 0 # Detectó pero leyó mal
    fallos_de_deteccion = 0   # No detectó nada en ese fotograma
    
    total_muestras = len(gt_df)
    
    # Iteramos sobre CADA fila de nuestro Ground Truth
    for _, fila_gt in gt_df.iterrows():
        frame_gt = fila_gt['fotograma']
        texto_gt = fila_gt['texto_correcto'].strip().upper()
        
        # Buscamos las detecciones del modelo para ESE fotograma
        detecciones_en_frame = modelo_df[modelo_df['fotograma'] == frame_gt]
        
        if detecciones_en_frame.empty:
            # El modelo no detectó NINGUNA matrícula en este fotograma
            fallos_de_deteccion += 1
            continue
            
        # Obtenemos una lista de todos los textos que detectó en ese frame
        textos_detectados = set(
            detecciones_en_frame['texto_matricula'].dropna().str.strip().str.upper()
        )
        
        if not textos_detectados:
             # Detectó la caja (tipo_objeto='matricula') pero el OCR falló (texto vacío)
            detecciones_incorrectas += 1
            continue

        if texto_gt in textos_detectados:
            # ¡Acierto! El texto correcto está entre los textos detectados
            aciertos_exactos += 1
        else:
            # Detectó matrículas, pero ninguna coincide con la correcta
            detecciones_incorrectas += 1
            
    # --- 4. CÁLCULO DE MÉTRICAS ---
    print("\n---" + ("-" * len(nombre_modelo)) + "---")
    print(f" RESULTADOS: {nombre_modelo}")
    print("---" + ("-" * len(nombre_modelo)) + "---")
    
    print(f"Total de muestras (Ground Truth): {total_muestras}")
    print(f"Aciertos Exactos:               {aciertos_exactos}")
    print(f"Errores (Lectura Incorrecta):   {detecciones_incorrectas}")
    print(f"Fallos (No Detección):          {fallos_de_deteccion}")
    
    # Calculamos la precisión (Precision) y la tasa de detección (Recall)
    # Precisión: De las que detectó, cuántas acertó.
    # Recall: De todas las que había, cuántas encontró (aunque sea mal).
    
    total_detectadas = aciertos_exactos + detecciones_incorrectas
    if total_detectadas > 0:
        precision = (aciertos_exactos / total_detectadas) * 100
        print(f"\nPrecisión (Aciertos / Detectadas): {precision:.2f}%")
    else:
        print("\nPrecisión: N/A (No hubo detecciones)")

    if total_muestras > 0:
        recall = (total_detectadas / total_muestras) * 100
        tasa_acierto_global = (aciertos_exactos / total_muestras) * 100
        print(f"Recall (Detectadas / Totales):     {recall:.2f}%")
        print(f"TASA DE ÉXITO GLOBAL (Aciertos / Totales): {tasa_acierto_global:.2f}%")

# --- 5. EJECUCIÓN ---
print("Iniciando comparativa de precisión...")

# Comparar EasyOCR
gt_easy, easy_df = cargar_datos(archivo_gt, archivo_easyocr)
if gt_easy is not None:
    comparar_modelo("EasyOCR", gt_easy, easy_df)

# Comparar Tesseract
gt_tess, tess_df = cargar_datos(archivo_gt, archivo_tesseract)
if gt_tess is not None:
    comparar_modelo("Tesseract", gt_tess, tess_df)

Iniciando comparativa de precisión...

-------------
 RESULTADOS: EasyOCR
-------------
Total de muestras (Ground Truth): 6
Aciertos Exactos:               0
Errores (Lectura Incorrecta):   6
Fallos (No Detección):          0

Precisión (Aciertos / Detectadas): 0.00%
Recall (Detectadas / Totales):     100.00%
TASA DE ÉXITO GLOBAL (Aciertos / Totales): 0.00%

---------------
 RESULTADOS: Tesseract
---------------
Total de muestras (Ground Truth): 6
Aciertos Exactos:               1
Errores (Lectura Incorrecta):   5
Fallos (No Detección):          0

Precisión (Aciertos / Detectadas): 16.67%
Recall (Detectadas / Totales):     100.00%
TASA DE ÉXITO GLOBAL (Aciertos / Totales): 16.67%
